In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [27]:
df=pd.read_csv('/content/ratings_Beauty.csv')

In [28]:
df.head(101)

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1.369699e+09
1,A3JM6GV9MNOF9X,0558925278,3.0,1.355443e+09
2,A1Z513UWSAAO0F,0558925278,5.0,1.404691e+09
3,A1WMRR494NWEWV,0733001998,4.0,1.382573e+09
4,A3IAAVS479H7M7,0737104473,1.0,1.274227e+09
...,...,...,...,...
96,A1BG8QW55XHN6U,1304651118,5.0,1.385424e+09
97,A6R426V4J7AOM,1304651347,5.0,1.385856e+09
98,A1BG8QW55XHN6U,1304665704,5.0,1.385597e+09
99,AZ4FR14IPKML7,1304665720,5.0,1.388966e+09


In [41]:
df=df.drop('Timestamp',axis=1)
LE=LabelEncoder()
df['UserId']=LE.fit_transform(df['UserId'])
# df['ProductId']=LE.fit_transform(df['ProductId'])

KeyError: ignored

In [42]:
df

,UserId,ProductId,Rating
0,700196,0,5.0
1,786777,1,3.0
2,302200,1,5.0
3,280924,2,4.0
4,775381,3,1.0
...,...,...,...
1930075,424261,228023,5.0
1930076,526489,228023,5.0
1930077,856675,228023,5.0
1930078,1062770,228023,5.0


In [43]:

train,test=train_test_split(df,test_size=0.2)

In [44]:
train

,UserId,ProductId,Rating
249001,105939,17366,5.0
758523,404934,67293,5.0
1491640,169877,152747,1.0
1092350,425006,103349,5.0
573191,231814,49207,4.0
...,...,...,...
852611,1118304,77024,5.0
716791,946949,63110,3.0
249071,698904,17374,4.0
1769895,41601,195047,4.0


In [45]:
no_of_rated_products_per_user = df.groupby(by='UserId')['Rating'].count().sort_values(ascending=False)
no_of_rated_products_per_user.head(25)

UserId
793523     380
378143     316
807298     313
577120     272
1046739    269
1037047    244
189512     238
239270     222
852067     221
803097     214
985817     203
413451     186
256025     165
1045971    163
1100262    157
875169     154
1162515    152
123489     151
971597     151
193837     151
451485     146
505355     140
602694     139
655747     136
927502     133
Name: Rating, dtype: int64

In [46]:
nearest_neighbors=NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=5)

In [47]:
nearest_neighbors

NearestNeighbors(algorithm='brute', metric='cosine')

In [48]:
nearest_neighbors.fit(train[['UserId','ProductId']])

NearestNeighbors(algorithm='brute', metric='cosine')

recommendation_function

In [65]:
import pandas as pd

def recommendation_system_function(userID, n_recommendation, nearest_neighbors):
    user_profile = df[df['UserId'] == userID].drop('Rating', axis=1)

    # Find similar users using nearest neighbors based on user profiles
    distances, indices = nearest_neighbors.kneighbors(user_profile)
    similar_users = indices.flatten()

    # Retrieve products rated by similar users
    user_recommendations = []
    for user in similar_users:
        products_rated_by_user = df[df['UserId'] == user]
        product_list = products_rated_by_user['ProductId'].unique()
        user_recommendations.extend(product_list)
    unique_recommendations = list(set(user_recommendations))

    # Calculate the number of ratings for each recommended product
    product_ratings_count = df[df['ProductId'].isin(unique_recommendations)]['ProductId'].value_counts()

    # Sort the products based on ratings count in descending order
    sorted_recommendations = product_ratings_count.sort_values(ascending=False)

    # Select the top-rated recommendations
    top_recommendations = sorted_recommendations.index[:n_recommendation]

    # Convert recommended product IDs back to original form
    recommended_product_codes = LE.inverse_transform(top_recommendations)

    # Create a DataFrame to store the recommended products and their data
    recommendation_table = pd.DataFrame({'Product ID': top_recommendations, 'Product Code': recommended_product_codes})
    recommendation_table['Rating Count'] = recommendation_table['Product ID'].map(product_ratings_count)
    recommendation_table.set_index('Product ID', inplace=True)

    # Print the recommendation table
    print("Recommended Products:")
    print(recommendation_table)

    return recommended_product_codes


In [66]:
print(df[df['ProductId']=='B003BMGA6C']['UserId'])

Series([], Name: UserId, dtype: int64)


In [68]:
user_id = 105  # Replace with the desired user ID
num_recommendations = 10  # Replace with the desired number of recommendations

recommended_products = recommendation_system_function(user_id, num_recommendations, nearest_neighbors)


Recommended Products:
           Product Code  Rating Count
Product ID                           
2906         B00014DZ2K           213
212382       B00C1F13CQ            94
209240       B00BOBKSLK            67
88948        B00319V9BG            21
73178        B0025FG3ZY             4
184672       B008Z72KUI             3


,UserId,ProductId,Rating
